In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os
import matplotlib.pyplot as plt

def detect_and_predict_mask(frame, faceNet, maskNet):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
        (104.0, 177.0, 123.0))

    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)

    faces = []
    locs = []
    preds = []

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")


            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)
            locs.append((startX, startY, endX, endY))

    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)

    return (locs, preds)

In [2]:
prototxtPath = r"face_detector\deploy.prototxt"
weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

maskNet = load_model("models\mask_detector_resnet_20.model")

In [ ]:
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()

count = 0
result = [0, 0, 0]

while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
    count += 1
    
    for (box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box
        
        length = (endX - startX)
        breadth = (endY - startY)
        
        if length < 150 and breadth < 230:
            cv2.putText(frame, "Please stand closer to the camera", (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            count -= 1
            break
        
        (incorrect_mask, mask, without_mask) = pred
        
        
        M = np.argmax(pred)
        if M == 0:
            label = "Incorrect Mask"
            color = (255, 0, 0)
            result[0] += 1
        elif M == 1:
            label = "Mask"
            color = (0, 255, 0)
            result[1] += 1
        else:
            label = "No Mask" 
            color = (0, 0, 255)
            result[2] += 1
        
        output = label + "," + str(length) + "," + str(breadth)
        #label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        label = "{}: {:.2f}%".format(output, pred[M] * 100)
        
        cv2.putText(frame, output, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    if count == 50:
        break


cv2.destroyAllWindows()
vs.stop()
result = np.array(result)
final_result = np.argmax(result)

import serial 
import time 
 
ArduinoSerial = serial.Serial('com3',9600) 
time.sleep(2) 
    
if (final_result == 1):
    var = '1'
    ArduinoSerial.write(var.encode())
    print ("LED turned ON")
    time.sleep(1)
    
else:
    var = '0'
    ArduinoSerial.write(var.encode())
    print ("LED turned OFF")
    time.sleep(1)

if final_result == 0:
    print("Incorrect Mask")
elif final_result == 1:
    print("With Mask")
else:
    print("Without Mask")

ArduinoSerial.close()